# Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

Problem Statement: Need to predict whether the given news is fake or not.

## Loading Dataset

In [1]:
import pandas as pd

## Reading CSV

This dataset contains some bad lines, hence we need to eliminate them during the read.

In [5]:
df = pd.read_csv('train.csv', on_bad_lines='warn', engine='python')

<ipython-input-5-994837006473>:1: ParserWarning: Skipping line 13535: field larger than field limit (131072)

  df = pd.read_csv('train.csv', on_bad_lines='warn', engine='python')
<ipython-input-5-994837006473>:1: ParserWarning: Skipping line 16297: unexpected end of data

  df = pd.read_csv('train.csv', on_bad_lines='warn', engine='python')
<ipython-input-5-994837006473>:1: ParserWarning: Skipping line 13541: Expected 5 fields in line 13541, saw 6

  df = pd.read_csv('train.csv', on_bad_lines='warn', engine='python')


In [6]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1.0
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0.0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1.0
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1.0
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1.0


In [7]:
df.shape

(16293, 5)

In [8]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1.0
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0.0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1.0
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1.0
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1.0


## Checking if dataset has null values

In [9]:
df.isnull().sum()

,0
id,0
title,427
author,1565
text,49
label,16


As we have some Null values lets drop the null values in the dataset as we cant replace them since its a Text data.

## Dropping Na Values

In [10]:
###Drop Nan Values

df = df.dropna()


In [11]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1.0
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0.0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1.0
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1.0
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1.0


## Gathering the Independent features (X)

In [13]:
## Get the Independent Features by dropping the output column

X = df.drop('label',axis=1)

## Gathering the Dependent feature (y)

In [15]:
## Get the Dependent features

y = df['label']

In [16]:
X.shape

(14303, 4)

In [17]:
y.shape

(14303,)

## Importing Tensorflow

In [18]:
import tensorflow as tf

In [19]:
tf.__version__

'2.18.0'

## Importing all the required libraries

In [20]:
from tensorflow.keras.layers import Embedding # used for Word2Vec implementation
from tensorflow.keras.preprocessing.sequence import pad_sequences # pre/post padding
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

## Specifying Vocabulary size

In [22]:
### Vocabulary size

voc_size = 5000

## Onehot Representation

Creating a copy of all the Independent features and we are not working directly on X as its our data.

In [23]:
messages = X.copy()

In [24]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [25]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
16287,16272,Iran Announces Opening of Underground Missile ...,John Hayward,Iran announced on Thursday that another underg...
16289,16274,Portland Bar Offers ’Free Whiskey for Life’ to...,Lucas Nolan,"A bar in Portland, Oregon has offered a lifeti..."
16290,16275,"With a Cuckoo’s Journey From China, a Mystery ...",Chris Buckley,BEIJING — When Flappy McFlapperson and Skyb...
16291,16276,La Superluna en imágenes,Tomás Fuentes,Por Tomás Fuentes \nEn la noche del lunes se p...


## Resetting the index

However its not required.

In [26]:
messages.reset_index(inplace=True)

In [27]:
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
14298,16287,16272,Iran Announces Opening of Underground Missile ...,John Hayward,Iran announced on Thursday that another underg...
14299,16289,16274,Portland Bar Offers ’Free Whiskey for Life’ to...,Lucas Nolan,"A bar in Portland, Oregon has offered a lifeti..."
14300,16290,16275,"With a Cuckoo’s Journey From China, a Mystery ...",Chris Buckley,BEIJING — When Flappy McFlapperson and Skyb...
14301,16291,16276,La Superluna en imágenes,Tomás Fuentes,Por Tomás Fuentes \nEn la noche del lunes se p...


## Preprocessing the Dataset

Since its a sentiment analysis we will remove stopwords and perform stemming operation.

### Importing nltk

In [28]:
import nltk
import re
from nltk.corpus import stopwords

### Downloading stopwords

In [29]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

### Stemming

In [30]:
### Dataset Preprocessing

from nltk.stem.porter import PorterStemmer # Stemming purpose

ps = PorterStemmer() # object for PorterStemmer class

corpus = [] # list to store preprocesed sentences

for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i]) # removing special characters
    review = review.lower() # lowering the sentences

    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)

    corpus.append(review)

In [31]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [32]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

## Deriving One hot representation

In [33]:
onehot_repr = [one_hot(words,voc_size) for words in corpus]

onehot_repr

[[1133, 1373, 1299, 109, 3968, 2155, 3704, 4750, 4271, 3242],
 [2964, 3476, 2241, 3882, 2370, 3216, 1102],
 [2828, 4216, 2908, 364],
 [2335, 4004, 1560, 4987, 2607, 2302],
 [2520, 2370, 1744, 1514, 4345, 2427, 2370, 298, 4085, 887],
 [4209,
  1771,
  1864,
  3344,
  3569,
  1922,
  3049,
  4504,
  3000,
  2893,
  1371,
  4071,
  3433,
  4930,
  1102],
 [512, 4337, 4897, 1282, 591, 4377, 4036, 2241, 4979, 2518, 1108],
 [4559, 2057, 3286, 2729, 2217, 2874, 1922, 2716, 4979, 2518, 1108],
 [3347, 801, 107, 3865, 4343, 1491, 926, 4549, 1922, 4571],
 [484, 143, 1829, 4376, 1069, 131, 2962, 4241],
 [759, 3291, 4254, 2741, 4882, 4249, 3692, 2478, 2899, 3309, 302],
 [4987, 1539, 3968, 1491, 1922, 2217],
 [1154, 4204, 3175, 4280, 1757, 850, 2880, 2447, 2896],
 [4738, 2870, 2884, 3188, 1078, 4447, 4018, 4979, 2518, 1108],
 [2080, 4243, 1882, 134, 1686, 4979, 2518, 1108],
 [4728, 3577, 1233, 3263, 2214, 2912, 2136, 3651, 3496, 4806],
 [97, 678, 3476],
 [2668, 499, 4455, 3675, 1922, 3760, 1076, 110

In [34]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [35]:
onehot_repr[1]

[2964, 3476, 2241, 3882, 2370, 3216, 1102]

## Applying padding

Providing maximum sentence length of 20.

In [38]:
sent_length = 20

embedded_docs = pad_sequences(onehot_repr, padding='post', maxlen=sent_length)

print(embedded_docs)

[[1133 1373 1299 ...    0    0    0]
 [2964 3476 2241 ...    0    0    0]
 [2828 4216 2908 ...    0    0    0]
 ...
 [4964 4367 4481 ...    0    0    0]
 [4132  142  256 ...    0    0    0]
 [3942 4683 1922 ...    0    0    0]]


In [39]:
embedded_docs[1]

array([2964, 3476, 2241, 3882, 2370, 3216, 1102,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [40]:
embedded_docs[0]

array([1133, 1373, 1299,  109, 3968, 2155, 3704, 4750, 4271, 3242,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

## Creating the model adding the Embedding and LSTM layer

In [41]:
## Creating model
embedding_vector_features = 40 # features representation (Hyperparameter can be adjusted)

model = Sequential()

model.add(Embedding(voc_size, # vocabulary size
                    embedding_vector_features, # 40 features representation (each index gets represented in 40 unique vectors)
                    input_length=sent_length)) # Adding Embedding layer

model.add(LSTM(100)) # LSTM with 100 neurons (neurons can be changed)

model.add(Dense(1,activation='sigmoid')) # Sigmoid because output is binary

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [42]:
len(embedded_docs),y.shape

(14303, (14303,))

In [43]:
import numpy as np

X_final = np.array(embedded_docs)

y_final = np.array(y)

In [44]:
X_final.shape,y_final.shape

((14303, 20), (14303,))

## Train Test Split

In [45]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

## Training the model

In [56]:
### Finally Training

model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=10, batch_size=64)

Epoch 1/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.7310 - loss: 0.4706 - val_accuracy: 0.9081 - val_loss: 0.2178
Epoch 2/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.9400 - loss: 0.1517 - val_accuracy: 0.9144 - val_loss: 0.2077
Epoch 3/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.9593 - loss: 0.1054 - val_accuracy: 0.9127 - val_loss: 0.2259
Epoch 4/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.9759 - loss: 0.0733 - val_accuracy: 0.9125 - val_loss: 0.2581
Epoch 5/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9815 - loss: 0.0502 - val_accuracy: 0.9095 - val_loss: 0.2935
Epoch 6/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.9861 - loss: 0.0402 - val_accuracy: 0.9093 - val_loss: 0.3374
Epoch 7/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9883 - loss: 0.0346 - val_accuracy: 0.9095 - val_loss: 0.3283
Epoch 8/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9910 - loss: 0.0267 - val_accuracy

## Adding Dropout

Dropout helps to improve the performance of the model. It will reduce Overfitting.

In [55]:
from tensorflow.keras.layers import Dropout

embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3)) # 30% of LSTM while training will be disabled
model.add(LSTM(100))
model.add(Dropout(0.3)) # 30% of LSTM while training will be disabled
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


## Prediction

In [57]:
y_pred = model.predict(X_test)

148/148 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


In [58]:
y_pred = np.where(y_pred > 0.6, 1, # Fake news
                  0) ## AUC ROC Curve

In [59]:
y_pred[0]

array([1])

## Performance Metrics And Accuracy

In [60]:
from sklearn.metrics import confusion_matrix

In [61]:
confusion_matrix(y_test,y_pred)

array([[2528,  162],
       [ 420, 1610]])

In [62]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.8766949152542373

In [63]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

         0.0       0.86      0.94      0.90      2690
         1.0       0.91      0.79      0.85      2030

    accuracy                           0.88      4720
   macro avg       0.88      0.87      0.87      4720
weighted avg       0.88      0.88      0.88      4720

